In [1]:
import os

cwd = os.path.abspath("")
cwd

'/Users/jskaf/BBLLM-Goodfire-Hackathon/src'

In [2]:
%cd ..
os.path.abspath("")

/Users/jskaf/BBLLM-Goodfire-Hackathon


/Users/jskaf/BBLLM-Goodfire-Hackathon/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/Users/jskaf/BBLLM-Goodfire-Hackathon'

In [3]:
THRESHOLD_EDGE = 3
PATH_GRAPH = os.path.join(os.path.abspath(""), "data", "graph_70B.weighted")
PATH_GRAPH

'/Users/jskaf/BBLLM-Goodfire-Hackathon/data/graph_70B.weighted'

In [4]:
import networkx as nx

G = nx.read_weighted_edgelist(PATH_GRAPH, nodetype=int)

In [5]:
print(len(G.nodes), len(G.edges))

23398 263983


In [6]:
nx.density(G)

0.0009644218561813173

In [7]:
G.remove_edges_from([(u, v) for u, v, w in G.edges(data=True) if w['weight'] < THRESHOLD_EDGE])

In [8]:
print(len(G.nodes), len(G.edges))

23398 17618


In [9]:
nx.density(G)

6.436469114375717e-05

In [10]:
nb_composantes = nx.number_connected_components(G)
nb_noeuds_total = G.number_of_nodes()
print(f'Nombre de composantes connectées : {nb_composantes}')
print(f'Nombre de composantes déconnectées : {nb_noeuds_total - nb_composantes}')
largest_cc = max(nx.connected_components(G), key=len)
sous_graphe = G.subgraph(largest_cc).copy() 

Nombre de composantes connectées : 13698
Nombre de composantes déconnectées : 9700


In [11]:
print(len(sous_graphe.nodes), len(sous_graphe.edges))

8925 16807


In [12]:
nx.density(sous_graphe)

0.0004220388289784762

In [16]:
MAPPING_FILE = os.path.join(os.path.abspath(""), "data", "index_label_mapping_70B.json")
import json

with open(MAPPING_FILE, 'r') as f:
    node_mapping = json.load(f)

mapping = {int(k): v for k, v in node_mapping.items()}
    
final_graph = nx.relabel_nodes(sous_graphe, mapping)

In [17]:
import ipysigma

ipysigma.Sigma(final_graph,   node_metrics={"community": {"name": "louvain", "resolution": 1.5}},
  node_color="community", edge_size = 'weight')

Sigma(nx.Graph with 8,704 nodes and 16,663 edges)

In [19]:
#Save the graph
nx.write_weighted_edgelist(sous_graphe, os.path.join(os.path.abspath(""), "data", "graph_filtered_70B.weighted"))